# Konto Mieszkaniowe

## Podsumowanie
* na pierwsze mieszkanie (z wyjątkami) dla osób od 13 do 45 roku życia.
* oszczędzanie 3-10 lat, potem max 5 lat na zakup mieszkania/nieruchomości.
* wymaga regularnego (min. 11 miesięcznych rat każdego roku, każda od 500-2000 zł).
* państwo gwarantuje premię (po spełnieniu warunków) w wysokości inflacji lub wzrostu cen mieszkań (większy z tych wskaźników), `1% < premia < 15%`
* dodatkowo bank wypłaca odsetki w wysokści co najmniej 75% standardowej ofery lokaty banku.  Od tych odsetek nie płacimy podatku Belki.
* max. jedno konto na osobę, przy czym konto jest prowadzone w imieniu dziecka a nie rodzica w przypadku niepełnoletnich dzieci.

## Podstawa Prawna
* Ustawa z dnia 26 maja 2023 r. o pomocy państwa w oszczędzaniu na cele mieszkaniowe ([Dz.U. poz. 1114](https://dziennikustaw.gov.pl/DU/2023/1114))
* Ustawa z dnia 1 października 2021 r. o rodzinnym kredycie mieszkaniowym i bezpiecznym kredycie 2% ([Dz. U. 2021 r. poz. 2133](https://isap.sejm.gov.pl/isap.nsf/DocDetails.xsp?id=WDU20210002133))

## Materiały
* [PierwszeMieszkanie.gov.pl](https://pierwszemieszkanie.gov.pl/)
* [gov.pl: Ministerstwo Rozwoju i Technologii: Konto Mieszkaniowe](https://www.gov.pl/web/rozwoj-technologia/konto-mieszkaniowe)
* [gov.pl: Ministerstwo Rozwoju i Technologii: Program Pierwsze Mieszkanie](https://www.gov.pl/web/rozwoj-technologia/pierwsze-mieszkanie)
* [bgk.pl: Konto Mieszkaniowe / Program Pierwsze Mieszkanie: O produkcie](https://www.bgk.pl/osoby-prywatne/mieszkalnictwo/konto-mieszkaniowe/#c28296)
* [bgk.pl: Konto Mieszkaniowe / Program Pierwsze Mieszkanie: Lista banków prowadzących Konto Mieszkaniowe](https://www.bgk.pl/osoby-prywatne/mieszkalnictwo/konto-mieszkaniowe/#c28304)
* [Subiektywnie o Finansach: W pierwszym banku już dziś założysz Konto Mieszkaniowe!](https://subiektywnieofinansach.pl/od-dzis-zalozysz-konto-mieszkaniowe-w-pierwszym-banki-warto/)
* [Marcin Iwuć: Konto Mieszkaniowe - czy to się opłaca?](https://marciniwuc.com/konto-mieszkaniowe/)

## Kod do wyliczenia szacowanych zysków

In [1]:
import pandas as pd

# TODO: 1% dla banku
# TODO: porównaj do EDO i do inflacji
# TODO: pierwszy rok bez premii jak nie ma 9 rat, art 14.1


def roczny_wskaznik_premii(inflacja, wzrost_m2):
    # większy z obu czynników (art. 14.3)
    wskaznik = max(inflacja, wzrost_m2)
    # wskaźnik nie przekracza 1% od dołu i 15% od góry (art. 14.4)
    return min(0.15, max(0.01, wskaznik))


def zalozenia_inflacji_i_wzrostu_m2(inflacja: list, wzrost_m2: list):
    df_inflacja = pd.DataFrame(data={
        'Inflacja': inflacja,
        'Wzrost m2': wzrost_m2
    },
        index=pd.date_range(
        start='2024',
        periods=len(inflacja),
        freq=pd.offsets.YearBegin(),
        name='Rok'
    ).strftime('%Y')
    )
    df_inflacja['Premia'] = df_inflacja.apply(
        lambda row: roczny_wskaznik_premii(row["Inflacja"], row["Wzrost m2"] or 0), axis=1)
    return df_inflacja


def formatuj_inflacje(df_inflacja: pd.DataFrame):
    styled = df_inflacja.style.format({
        'Inflacja': '{:,.2%}'.format,
        'Premia': '{:,.2%}'.format,
    })
    return styled


def oblicz_skladnik_naliczeniowy(suma_wplat=float, premia=float):
    return suma_wplat * premia / 12  # Art. 14.2


def symulacja_konta(data_startu, ile_lat, ile_wplat, wysokosc_wplat, premia):
    df_konto = pd.DataFrame(data={
        'Miesiąc': pd.date_range(
            start=data_startu,
            periods=ile_wplat,
            freq=pd.offsets.MonthBegin()
        ),
        'Wpłata': [wysokosc_wplat] * ile_wplat
    },
        index=pd.Index(range(1, ile_wplat+1), name="Wpłata nr")
    )
    df_konto['Suma Wpłat'] = df_konto['Wpłata'].cumsum()
    df_konto['Rok'] = pd.DatetimeIndex(df_konto['Miesiąc']).year
    df_konto['Składnik Naliczeniowy'] = df_konto.apply(
        lambda row: oblicz_skladnik_naliczeniowy(
            suma_wplat=row['Suma Wpłat'], premia=premia._get_value(
                str(row['Rok']), 'Premia')
        ), axis=1
    )
    df_roczne = df_konto[['Wpłata', 'Składnik Naliczeniowy', 'Rok']].groupby(
        'Rok').sum()
    df_roczne = df_roczne.rename(columns={
        "Składnik Naliczeniowy": "Premia roczna",
        "Wpłata": "Suma Wpłat"
    })
    df_roczne['Wpłata całkowita'] = df_roczne['Suma Wpłat'].cumsum()
    df_roczne['Premia całkowita'] = df_roczne['Premia roczna'].cumsum()

    df_konto_styled = df_konto.style.format({
        'Miesiąc': lambda d: d.strftime('%Y-%m'),
        'Wpłata': '{:,.0f} zł'.format,
        'Suma Wpłat': '{:,.0f} zł'.format,
        'Składnik Naliczeniowy': '{:,.2f} zł'.format
    })
    df_roczne_styled = df_roczne.style.format({
        'Premia roczna': '{:,.0f} zł'.format,
        'Premia całkowita': '{:,.0f} zł'.format,
        'Suma Wpłat': '{:,.0f} zł'.format,
        'Wpłata całkowita': '{:,.0f} zł'.format,
    })
    return df_konto_styled, df_roczne_styled

## Test symulatora
Liczymy scenariusz ze stron rządowych [PierwszeMieszkanie.gov.pl](https://pierwszemieszkanie.gov.pl/), a konkretnie ten scenariusz:
<img src="https://www.gov.pl/photo/d64c123f-f6b6-4e32-982c-ae85680f408d" width="500"/>
Czyli:
* 3 lata oszczędzania
* po 1000zł miesięcznie
* przy 9.6% inflacji przez wszystkie 3 lata, wskaźnik wzrostu cen m2 nieistotny


In [2]:
df_inflacja1 = zalozenia_inflacji_i_wzrostu_m2(
    inflacja=[pct/100 for pct in [9.6] * 3], wzrost_m2=None)
display(formatuj_inflacje(df_inflacja1))
df_konto1, df_roczne1 = symulacja_konta(
    data_startu='2024-01', ile_lat=3, ile_wplat=3*12, wysokosc_wplat=1000, premia=df_inflacja1)
display(df_roczne1)
display(df_konto1)

,Inflacja,Wzrost m2,Premia
Rok,,,
2024,9.60%,None,9.60%
2025,9.60%,None,9.60%
2026,9.60%,None,9.60%


,Suma Wpłat,Premia roczna,Wpłata całkowita,Premia całkowita
Rok,,,,
2024,"12,000 zł",624 zł,"12,000 zł",624 zł
2025,"12,000 zł","1,776 zł","24,000 zł","2,400 zł"
2026,"12,000 zł","2,928 zł","36,000 zł","5,328 zł"


,Miesiąc,Wpłata,Suma Wpłat,Rok,Składnik Naliczeniowy
Wpłata nr,,,,,
1,2024-01,"1,000 zł","1,000 zł",2024,8.00 zł
2,2024-02,"1,000 zł","2,000 zł",2024,16.00 zł
3,2024-03,"1,000 zł","3,000 zł",2024,24.00 zł
4,2024-04,"1,000 zł","4,000 zł",2024,32.00 zł
5,2024-05,"1,000 zł","5,000 zł",2024,40.00 zł
6,2024-06,"1,000 zł","6,000 zł",2024,48.00 zł
7,2024-07,"1,000 zł","7,000 zł",2024,56.00 zł
8,2024-08,"1,000 zł","8,000 zł",2024,64.00 zł
9,2024-09,"1,000 zł","9,000 zł",2024,72.00 zł


## Założenia inflacji i wzrostu cen nieruchomości

Prognozy inflacji oparte o [Polski Instytut Ekonomiczny: Struktura inflacji CPI – prognoza PIE](https://pie.net.pl/prognozy/) i długoterminowy cel inflacyjny NBP (2.5%).

Dla prostoty na razie nie uwzględniam wzrostu cen nieruchomości, który w niektórych latach może przekraczać inflację i zwiększyłby wysokość premii mieszkaniowej.

## Symulacja oszczędzania

In [3]:
df_inflacja2 = zalozenia_inflacji_i_wzrostu_m2(
    inflacja=[pct/100 for pct in [7.6, 4.3] + [2.5] * 8], wzrost_m2=None)
display(formatuj_inflacje(df_inflacja2))
df_konto2, df_roczne2 = symulacja_konta(
    data_startu='2024-01', ile_lat=10, ile_wplat=10*12, wysokosc_wplat=1000, premia=df_inflacja2)
display(df_roczne2)
display(df_konto2)

,Inflacja,Wzrost m2,Premia
Rok,,,
2024,7.60%,None,7.60%
2025,4.30%,None,4.30%
2026,2.50%,None,2.50%
2027,2.50%,None,2.50%
2028,2.50%,None,2.50%
2029,2.50%,None,2.50%
2030,2.50%,None,2.50%
2031,2.50%,None,2.50%
2032,2.50%,None,2.50%


,Suma Wpłat,Premia roczna,Wpłata całkowita,Premia całkowita
Rok,,,,
2024,"12,000 zł",494 zł,"12,000 zł",494 zł
2025,"12,000 zł",795 zł,"24,000 zł","1,290 zł"
2026,"12,000 zł",762 zł,"36,000 zł","2,052 zł"
2027,"12,000 zł","1,062 zł","48,000 zł","3,114 zł"
2028,"12,000 zł","1,362 zł","60,000 zł","4,477 zł"
2029,"12,000 zł","1,662 zł","72,000 zł","6,140 zł"
2030,"12,000 zł","1,962 zł","84,000 zł","8,102 zł"
2031,"12,000 zł","2,262 zł","96,000 zł","10,364 zł"
2032,"12,000 zł","2,562 zł","108,000 zł","12,927 zł"


,Miesiąc,Wpłata,Suma Wpłat,Rok,Składnik Naliczeniowy
Wpłata nr,,,,,
1,2024-01,"1,000 zł","1,000 zł",2024,6.33 zł
2,2024-02,"1,000 zł","2,000 zł",2024,12.67 zł
3,2024-03,"1,000 zł","3,000 zł",2024,19.00 zł
4,2024-04,"1,000 zł","4,000 zł",2024,25.33 zł
5,2024-05,"1,000 zł","5,000 zł",2024,31.67 zł
6,2024-06,"1,000 zł","6,000 zł",2024,38.00 zł
7,2024-07,"1,000 zł","7,000 zł",2024,44.33 zł
8,2024-08,"1,000 zł","8,000 zł",2024,50.67 zł
9,2024-09,"1,000 zł","9,000 zł",2024,57.00 zł
